In [6]:
!pip install opstrat
!pip install numpy-financial
!pip install mibian

import numpy as np
from scipy.stats import norm
import pandas as pd
import mibian
import opstrat as op
import numpy_financial as npf
from scipy.optimize import minimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Options

# Here we are calculating expectation instead of Monte Carlo approximation leading to d1 and d2.
# There is no simulation of price paths involved, it is an explicit formula, making it simple to use, 
# However, it cannot be adapted for other pricing methods such as: Merton Jump-Diffusion.

def BS_EP_call(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)

    V = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return V 


def ds(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
    return (d1, d2)

def normds(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    return (Nd1, Nd2)

In [4]:
# Black-Scholes Explicit Pricing Put:

def BS_EP_put(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)

    V = K*(np.exp(-r*T))*norm.cdf(-d2) - S0*norm.cdf(-d1)  
    return V
    
def ds(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
    return (d1, d2)

def normnegds(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
    Nd1 = norm.cdf(-d1)
    Nd2 = norm.cdf(-d2)
    return (Nd1, Nd2)

In [5]:
S0 = 17 
K = 17
T = 0.25
sig = 0.15 # Vix Used, where VIX = 100*market volatility.
r = 0.0175 # 3 month T-Bills. 
print("Black-Scholes Explicit Call Price: " + str(BS_EP_call(S0, K, T, r, sig)) + " ds: " + str(ds(S0, K, T, r, sig)) + " Nds: "+ str(normds(S0, K, T, r, sig)))
print("Black-Scholes Explicit Put Price: " + str(BS_EP_put(S0, K, T, r, sig)) + " ds: " + str(ds(S0, K, T, r, sig)) + " Nds: " + str(normnegds(S0, K, T, r, sig)))

Black-Scholes Explicit Call Price: 0.5453911755755314 ds: (0.09583333333333334, 0.020833333333333343) Nds: (0.5381735284917541, 0.5083106963251719)
Black-Scholes Explicit Put Price: 0.47117863388331305 ds: (0.09583333333333334, 0.020833333333333343) Nds: (0.4618264715082458, 0.4916893036748281)


In [7]:
# Implied Volatility Algorithm:

S0 = 350 
K = 330
T = 1 
r = 0.15
sig = 0.21 # Vix today
C = 71.52168469177215

# Generate a function for the explicit pricing B-S call so that it can be used in the algorithm.
def BS_EP_call(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)

    V = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return V 



# Generate a function for d1 and d2 so that they can be used in the algortithm.
def ds(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
    return (d1, d2)

d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
d2 = d1 - sig*np.sqrt(T)
d1_pdf = np.exp(-d1**2/2)*1/np.sqrt(np.pi*2)
    
# print(ds(S0, K, T, r, sig))
# print(BS_EP_call(S0, K, T, r, sig))

tol = 0.0001 # Tolerance level for error (Garret, 2015)
change = 1 # Percentage change in implied volatility across iterations
count = 0
max_it = 10000 # Maximum iterations
vol = 0.15 # Initial estimate for volatility
   
while change > tol: # Sets parameter for tolerance of incremental changes in 
    count += 1 
    vol0 = vol
    d1, d2 = ds( S0, K, T, r, vol)
    Target = BS_EP_call(S0, K, T, r, vol) - C # Target function of root finding.
    Vega = S0*np.sqrt(T)*d1_pdf # Call function differentiated partially with respect to Vega.
    vol = -Target/Vega + vol # Volatility necessary for root or IV.
    change = np.abs((vol-vol0)/vol0) # generating percentage change in volatility.
    if count > max_it:
        break; # Stops iterations once maximum reached.

print(("Implied Volatility is: %f, obtained after %d iterations.") % (vol, count)) 

Implied Volatility is: 0.210000, obtained after 4 iterations.


In [8]:
S0 = 350 
K = 330
T = 1 
r = 0.15
sig = 0.21 # Vix 
C = 71.52168469177215
c = mibian.BS([350, 330, 15, 365], callPrice= 71.52168469177215)
print(c.impliedVolatility)

21.000000648200512


In [9]:
p = mibian.BS([350, 330, 15, 365], putPrice= 5.55531691)
print(p.impliedVolatility)

20.999999716877937


In [11]:
K=17   #spot price
St=17   #current stock price
r=1.75     #4% risk free rate
t=91.2501    #time to expiry, 30 days 
v=15     #volatility 
type='c' #Option type call
#Black Scholes Model
bsm_c=op.black_scholes(K=K, St=St, r=r, t=t, 
                     v=v, type='c')
print(bsm_c['value'])
print(bsm_c['greeks'])

{'option value': 0.5453914942615725, 'intrinsic value': 0, 'time value': 0.5453914942615725}
{'delta': 0.5381735493447947, 'gamma': 0.31146220552887083, 'theta': -0.003186859658921205, 'vega': 0.03375475351566155, 'rho': 0.021508920682893937}


In [12]:
K=330    #spot price
St=350   #current stock price
r=15      #risk free rate
t=365     #time to expiry
v=21     #volatility 
type='p' #Option type put
#Black Scholes Model
bsm_p=op.black_scholes(K=K, St=St, r=r, t=t, 
                     v=v, type='p')
print(bsm_p['value'])
print(bsm_p['greeks'])

{'option value': 5.555316912041256, 'intrinsic value': 0, 'time value': 5.555316912041256}
{'delta': -0.13577968785034716, 'gamma': 0.0029656753333323713, 'theta': -0.00013405105698078573, 'vega': 0.7629199794997524, 'rho': -0.5307820765966277}


In [13]:
# Risk Neutral Probability Measure:
r = 0.04
u = 1.2
pu = 0.6
d = 1/u
pd = 0.4
dt = 1/12

Q = (np.exp(r*dt)-d)/(u-d)
round(Q,6)

0.463652

In [14]:
# Futures 
S0 = 350 
D = 0
t = 365 
r = 0.15
C = 10
Futs = (S0 - D + C)*(1 + r)**t
round(Futs,6)

5.140604947079175e+24

In [15]:
# Forward
r = 0.15 
S0 = 350 
q = 0.01
t = 365 
F = S0*np.exp(r-q*t) 
round(F,6)

10.569084

In [16]:
# EWMA and GARCH Models:
S_t = 43
S_tt = 42 #S_t-1
U_nn = np.log(S_t/S_tt) #U_n-1
o_nn = 0.013 #o_n-1

# EWMA
l = 0.94 
var_ewma = (l*o_nn**2)+(1-l)*U_nn**2
ewma = np.sqrt(var_ewma).round(8)

# GARCH
beta = 0.93
alpha = 0.05
omega = 0.000004
var_garch = omega+alpha*(U_nn)**2+beta*(o_nn)**2
garch = np.sqrt(var_garch).round(8)

# GJR-GARCH
d = 1 # 1 if change is negative, 0 if change is positive
var_gjr = omega + (alpha + alpha*d)*U_nn**2+beta*(o_nn**2)
gjr_garch = np.sqrt(var_gjr).round(8)

print(ewma, garch, gjr_garch)

0.01385933 0.01374242 0.01471524


In [19]:
# Hedge Ratios

Nf = 3 # No. Index unites traded in the futures market
Ns = 4 # No. Index units held
h_Naiive = -Nf/Ns # Hedge Ratio
print(h_Naiive)

# MVHR
Rf = [0.01, 0.03, 0.01, 0.015, 0.023]
Rs = [0.012, 0.031, 0.042, 0.013, 0.011]
sigf = np.std(Rf)
sigs = np.std(Rs)
cov = np.cov(Rf, Rs)[0][1]
h_MVHR = -cov*(sigf/sigs)
print(h_MVHR)

-0.75
6.244056959808052e-08


In [20]:
# Hazard Rate - The amount the bondholder expects to lose is priced into the bond yield.
y = 0.045 # bond yield above rf
R = 0.55 # Expected recovery rate
h = y/(1-R)
h

0.1

In [21]:
# Bond Pricing Approach
c = np.array([3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 103.5])
t = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])
r = 0.08
rdf = 0.04
cfs = [x*np.exp(-r*g) for x,g in zip(c, t)]
cfdf = [x*np.exp(-rdf*g) for x,g in zip(c, t)]
B = np.sum(cfs)
Bdf = np.sum(cfdf)
eloss = Bdf-B # Expected loss over the  5 year life of the corporate bond
print(B, Bdf, eloss)

95.30590681957425 113.27902973392891 17.97312291435466


In [22]:
# Contingent Claims Analysis - Merton - Provides Accurate Ranking of Credit Default Risk

def BS_EP_call(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)

    V = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return V 


S0 = 100 # Value of Firm
K = 80 # Face Value of Bonds
T = 1
sig = 0.3 # Volatility of Firm Value
r = 0.08 # rf

E = BS_EP_call(S0, K, T, r, sig)
D = S0-E # Value fo Debt
R = np.log(K/D) # Required rate of return on Debt when rf = r
print("Value of Firm Equity: " + str(BS_EP_call(S0, K, T, r, sig)) + " ds: " + str(ds(S0, K, T, r, sig)) + " Nds: "+ str(normds(S0, K, T, r, sig)), R)

# As debt increases constitution of capital structure, financial risk increases,
# along with probability of default, thus, raising the required rate of return to compensate for risk.

# Fair Value of the Guarantee on Debt:
G = (K*np.exp(-r)-D).round(2)
print("Value of Guarantee: " + str(G))

def BS_EP_put(S0, K, T, r, sig):
    d1 = (np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)

    V = K*(np.exp(-r*T))*norm.cdf(-d2) - S0*norm.cdf(-d1)  
    return V


# Risk neutral probability company will default on debt = N(-d2)
d2 = d1 - sig*np.sqrt(T)
Default_risk = norm.cdf(-d2)
print("Risk of company defaulting: " + str(Default_risk))


# Expected Loss on Debt:
elossd = (K*np.exp(-r)-D)/K*np.exp(-r)
erecov = (Default_risk-elossd)/Default_risk
print ("Expected recovery in the event of default: " + str(erecov))

Value of Firm Equity: 28.241078154036607 ds: (1.1604785043806993, 0.8604785043806993) Nds: (0.8770729797166348, 0.8052373361493941) 0.10871444147930563
Value of Guarantee: 2.09
Risk of company defaulting: 0.21200629968770274
Expected recovery in the event of default: 0.8862257062025072


In [23]:
# Weather, Energy and Insurance Derivatives
# Daily Weather Derivative:
cm = 100 # contract multiple
dhigh = 70 # daily high 
dlow = 44 # daily low
A = (dhigh+dlow)/2 # average heating/cooling days
HDD = 65 - A if A<65 else 0 # hdd payoff
CDD = A - 65 if A>65 else 0 # cdd payoff


In [24]:
# Call option written on HDD at Chicago O'Hare:
K = 700
r = 0.03
T = 1
payment = 10000 # payment per degree day
S0 = 710 # mean HDD from one month using historical data 
sig = 0.07 # stdev of ln(hdd)
d1 = (np.log(S0/K)+sig**2/2)/sig
d2 = (np.log(S0/K)-sig**2/2)/sig
Exp_payoff = payment*(S0*norm.cdf(d1)-K*norm.cdf(d2))
C = Exp_payoff*np.exp(-r*T)
print(d1, d2, Exp_payoff, C)

0.23763764274223398 0.167637642742234 250863.605011391 243449.46501318153


In [25]:
# Calamity Insurance Co.
AP = 15000 # Annual Premium
FAD = 10000 # Fixed Amount Deductible
MC = 300000 # Maximum Coverage
homes = 100000
# No Hurricane
NH_profit = AP*homes
# Hurricane 
H_loss = (MC-FAD-AP)*homes
# With Reinsurance
RI = 3000 # Reinsurance covers losses over $100k per household, costing $3k per home.
H_RI_loss = (100000-FAD-(AP-RI))*homes
NH_RI_profit = (AP-RI)*homes
print(NH_profit, H_loss, H_RI_loss, NH_RI_profit)

1500000000 27500000000 7800000000 1200000000


In [26]:
iy = 4000
oy = 0.008
oy2 = oy**2
w = 0.000005
a = 0.008
g = 0.04
b = 0.92
itu = 4050
itd = 3950
iytu = np.log(itu/iy) #0.01242251999855711
iytd = np.log(itd/iy) #-0.012578782206860073
iytu2 = iytu**2
iytd2 = iytd**2
o2tu = w+(a+g*0)*iytu2+(b*oy2)
otu = np.sqrt(o2tu)
o2td = w+(a+g*1)*iytu2+(b*oy2)
otd = np.sqrt(o2td)
print(otu*100, otd*100)
print(otu/oy, otd/oy)

0.8069358836048649 0.8443181399774522
1.008669854506081 1.0553976749718152


In [27]:
# VaR Single Asset
V = 10000000
x = 99
o = 0.02
o_d = V*o
days = 10
N = norm.ppf(0.99)
one_day_VaR = N*o_d
ten_day_VaR = one_day_VaR*np.sqrt(days)
print("We are " + str(x) + "% certain we will not lose more than $" + str(ten_day_VaR) + " in the next " + str(days) + " days.")

We are 99% certain we will not lose more than $1471311.5823719108 in the next 10 days.


In [28]:
# VaR Two Assets:
x = 0.99
N = norm.ppf(x)
V_a = 10000000
V_b = 5000000
o_a = 0.02
o_b = 0.01
o_ad = V_a*o_a
o_bd = V_b*o_b
rho = 0.3
o_ab = np.sqrt(o_ad**2+o_bd**2+2*rho*o_ad*o_bd)
VaR_one = N*o_ab
VaR_ten = VaR_one*np.sqrt(10)
undiv = (o_ad*N*np.sqrt(10)+o_bd*N*np.sqrt(10))
BOD = undiv-VaR_ten
print("We are " + str(x*100) + "% certain we will not lose more than $" + str(VaR_ten) + " in the next 10 days.") 
print("For which, the benefits of diversification are $" + str(BOD) + ".")

We are 99.0% certain we will not lose more than $1620113.8228721323 in the next 10 days.
For which, the benefits of diversification are $219025.65509275626.


In [29]:
# VaR Three Assets: 
V_c = 12000000.00
V_s = 9500000.00
V_g = 10250000.00

o_c = 0.0075
o_s = 0.0105
o_g = 0.0080

rho_gs = 0.73
rho_gc = 0.68
rho_sc = 0.82

x = 0.975
days = 5

o_cd = V_c*o_c
o_sd = V_s*o_s
o_gd = V_g*o_g 

N = norm.ppf(x)
N_inv = 1-norm.cdf(N)

print("There is a " + str((N_inv*100).round(2)) + "% probability that a normally distributed variable will decrease in value by more than " + str(N.round(2)) + " standard deviations.")

undiv = ((o_gd*N*np.sqrt(days))+(o_sd*N*np.sqrt(days))+(o_cd*N*np.sqrt(days))).round(2)
o_csg = (np.sqrt((o_cd**2)+(o_sd**2)+(o_gd**2)+(2*((rho_gs*o_gd*o_sd)+(rho_gc*o_gd*o_cd)+(rho_sc*o_sd*o_cd))))).round(2)
VaR_one = (o_csg*N).round(2)
VaR_days = (VaR_one*np.sqrt(days)).round(2)
BOD = (undiv-VaR_days).round(2)

print("We are " + str(x*100) + "% certain we will not lose more than $" + str(VaR_days.round(2)) + " over the next " + str(days) + " days.")
print("For which, the benefits of diversification are $" + str(BOD.round(2)) + ".")

There is a 2.5% probability that a normally distributed variable will decrease in value by more than 1.96 standard deviations.
We are 97.5% certain we will not lose more than $1086640.82 over the next 5 days.
For which, the benefits of diversification are $104334.18.
